In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Install dependencies
!pip install timm opencv-python matplotlib pillow --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 77.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install mmengine --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.7/452.7 kB 12.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 105.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 15.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires num

In [3]:
!pip install mmengine mmcv-lite --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.3/732.3 kB 12.3 MB/s eta 0:00:00a 0:00:01


In [4]:
# ============================================================
# Kaggle-Optimized Metric3D Evaluation on KITTI
# ============================================================

import torch
import cv2
import numpy as np
from PIL import Image
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from pathlib import Path
import time
import json
from tqdm import tqdm


class Metric3DKITTITester:
    def __init__(self, hub_model='metric3d_vit_small', device='cuda'):
        """
        Kaggle-optimized Metric3D evaluation.
        hub_model options:
            - metric3d_convnext_tiny   (CPU safe)
            - metric3d_convnext_large
            - metric3d_vit_small       (best on GPU)
            - metric3d_vit_large
            - metric3d_vit_giant2
        """
        self.hub_model = hub_model
        self.device = torch.device(device if torch.cuda.is_available() else "cpu")
        self.model = None

        print(f"🚀 Using device: {self.device}")
        print(f"📦 Model: {self.hub_model}")

    # ---------------------- MODEL LOADING ----------------------
    def load_model(self):
        print("📥 Loading Metric3D via PyTorch Hub...")

        try:
            self.model = torch.hub.load(
                'yvanyin/metric3d',
                self.hub_model,
                pretrain=True,
                trust_repo=True
            ).to(self.device)

            self.model.eval()
            print("✅ Metric3D loaded successfully!")
            return True

        except Exception as e:
            print(f"❌ Error loading model: {e}")
            return False

    # ---------------------- KITTI DEPTH LOADING ----------------------
    def load_kitti_depth(self, depth_path):
        depth = cv2.imread(str(depth_path), cv2.IMREAD_ANYDEPTH).astype(np.float32)
        depth /= 256.0
        valid_mask = depth > 0
        return depth, valid_mask

    # ---------------------- PREDICTION ----------------------
    def predict_depth(self, image_path):
        """GPU-accelerated inference with AMP for ViT models"""
        start_time = time.time()

        image = Image.open(str(image_path)).convert("RGB")
        img_np = np.array(image)

        img = torch.from_numpy(img_np).permute(2, 0, 1).float() / 255.
        img = img.unsqueeze(0).to(self.device)

        with torch.no_grad(), torch.cuda.amp.autocast(enabled=True):
            pred_depth, confidence, out_dict = self.model.inference({"input": img})
            pred_depth = pred_depth[0, 0].detach().cpu().numpy()

        return pred_depth, time.time() - start_time

    # ---------------------- METRIC COMPUTATION ----------------------
    def compute_metrics(self, pred, gt, valid):
        pred = pred[valid]
        gt = gt[valid]

        pred = np.clip(pred, 1e-3, None)
        gt = np.clip(gt, 1e-3, None)

        thresh = np.maximum(gt / pred, pred / gt)

        return {
            "delta_1": float((thresh < 1.25).mean()),
            "delta_2": float((thresh < 1.25 ** 2).mean()),
            "delta_3": float((thresh < 1.25 ** 3).mean()),
            "rmse": float(np.sqrt(((pred - gt) ** 2).mean())),
            "rmse_log": float(np.sqrt(((np.log(pred) - np.log(gt)) ** 2).mean())),
            "mae": float(np.abs(pred - gt).mean()),
            "abs_rel": float((np.abs(pred - gt) / gt).mean()),
            "sq_rel": float((((pred - gt) ** 2) / gt).mean())
        }

    # ---------------------- VISUALIZATION ----------------------
    def visualize_result(self, image_path, pred_depth, gt_depth, valid_mask, metrics, output_path):
        image = np.array(Image.open(image_path).convert('RGB'))

        fig, axes = plt.subplots(2, 2, figsize=(15, 10))

        # Input Image
        axes[0, 0].imshow(image)
        axes[0, 0].set_title('Input Image')
        axes[0, 0].axis('off')

        # Pred Depth
        im1 = axes[0, 1].imshow(pred_depth, cmap='plasma', vmin=0, vmax=80)
        axes[0, 1].set_title('Metric3D Depth')
        axes[0, 1].axis('off')
        plt.colorbar(im1, ax=axes[0, 1], fraction=0.046, label='meters')

        # GT
        gt_disp = gt_depth.copy()
        gt_disp[~valid_mask] = np.nan
        im2 = axes[1, 0].imshow(gt_disp, cmap='plasma', vmin=0, vmax=80)
        axes[1, 0].set_title('Ground Truth')
        axes[1, 0].axis('off')
        plt.colorbar(im2, ax=axes[1, 0], fraction=0.046, label='meters')

        # Error
        err = np.abs(pred_depth - gt_depth)
        err[~valid_mask] = np.nan
        im3 = axes[1, 1].imshow(err, cmap='hot', vmin=0, vmax=10)
        axes[1, 1].set_title('Absolute Error')
        axes[1, 1].axis('off')
        plt.colorbar(im3, ax=axes[1, 1], fraction=0.046, label='meters')

        # Metric text
        text = '\n'.join([f'{k}: {v:.4f}' for k, v in metrics.items() if k != 'inference_time'])
        fig.text(0.02, 0.02, text, fontsize=10, family='monospace',
                 bbox=dict(boxstyle='round', facecolor='wheat'))

        plt.tight_layout()
        plt.savefig(output_path, dpi=150, bbox_inches='tight')
        plt.close()

    # ---------------------- MAIN KITTI LOOP ----------------------
    def test_on_kitti(self, image_dir, gt_dir, output_dir, num_samples=20):
        image_dir = Path(image_dir)
        gt_dir = Path(gt_dir)
        output_dir = Path(output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)

        image_files = sorted(image_dir.glob("*.png"))
        image_files = image_files[:num_samples]

        print(f"📁 Evaluating {len(image_files)} images...\n")
        all_metrics = []

        for idx, img_path in tqdm(list(enumerate(image_files)), total=len(image_files)):
            pred, t = self.predict_depth(img_path)

            gt_path = gt_dir / img_path.name
            gt_depth, mask = self.load_kitti_depth(gt_path)

            pred = cv2.resize(pred, (gt_depth.shape[1], gt_depth.shape[0]))

            metrics = self.compute_metrics(pred, gt_depth, mask)
            metrics["inference_time"] = t
            all_metrics.append(metrics)

            vis_path = output_dir / f"result_{idx:03d}_{img_path.stem}.png"
            self.visualize_result(img_path, pred, gt_depth, mask, metrics, vis_path)

        # Save metrics.json
        avg = {k: float(np.mean([m[k] for m in all_metrics])) for k in all_metrics[0]}
        results_path = output_dir / "metrics.json"

        json.dump({
            "average": avg,
            "per_image": all_metrics
        }, open(results_path, "w"), indent=2)

        print(f"\n💾 Saved metrics to: {results_path}")


# =====================================================
# MAIN (Kaggle paths)
# =====================================================

IMAGE_DIR = "/kaggle/input/images-test"
GT_DIR = "/kaggle/input/ground-truth"
OUTPUT_DIR = "/kaggle/working/metric3d_results"
NUM_SAMPLES = 20

tester = Metric3DKITTITester(
    hub_model="metric3d_vit_small",
    device="cuda"
)

if tester.load_model():
    tester.test_on_kitti(IMAGE_DIR, GT_DIR, OUTPUT_DIR, NUM_SAMPLES)

print("\n✅ Done!")


🚀 Using device: cuda
📦 Model: metric3d_vit_small
📥 Loading Metric3D via PyTorch Hub...


Using cache found in /root/.cache/torch/hub/yvanyin_metric3d_main
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may

✅ Metric3D loaded successfully!
📁 Evaluating 20 images...



  0%|          | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_47/3172600041.py:74: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=True):
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
100%|██████████| 20/20 [00:29<00:00,  1.49s/it]


💾 Saved metrics to: /kaggle/working/metric3d_results/metrics.json

✅ Done!


In [6]:
import shutil
from pathlib import Path

OUTPUT_DIR = "/kaggle/working/metric3d_results"
ZIP_PATH = "/kaggle/working/metric3d_results"

print("📦 Creating ZIP file...")

# Create ZIP of entire directory
shutil.make_archive(ZIP_PATH, 'zip', OUTPUT_DIR)

print(f"✅ ZIP created at: {ZIP_PATH}.zip")
print("You can now download the file from the Kaggle sidebar (Output > Files).")



📦 Creating ZIP file...
✅ ZIP created at: /kaggle/working/metric3d_results.zip
You can now download the file from the Kaggle sidebar (Output > Files).
